# Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

# Standard library
from pathlib import Path
import pprint

# Third-party libraries
import duckdb
import fireducks.pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import optuna
from matplotlib import rcParams

rcParams["text.usetex"] = True

# Custom osbad library for anomaly detection
import osbad.config as bconf
import osbad.hyperparam as hp
import osbad.modval as modval
import osbad.viz as bviz
from osbad.database import BenchDB
from osbad.model import ModelRunner

# Import dataset

In [ ]:
# Define a global variable to save fig output
# The definition of PIPELINE_OUTPUT_DIR can be
# found in config.py
PIPELINE_OUTPUT_DIR = bconf.PIPELINE_OUTPUT_DIR

# Path to database directory
DB_DIR = bconf.DB_DIR

# Path to the DuckDB instance:
# "osbad/database/train_dataset_severson.db"
db_filepath = (
    DB_DIR.joinpath("train_dataset_severson.db"))

## Get unique cell labels from training dataset

In [ ]:
# Create a DuckDB connection
con = duckdb.connect(
    db_filepath,
    read_only=True)

# Load all training dataset from duckdb
df_duckdb = con.execute(
    "SELECT * FROM df_train_dataset_sv").fetchdf()

unique_cell_index_train = df_duckdb["cell_index"].unique()
print(unique_cell_index_train)

In [ ]:
training_cell_count = len(unique_cell_index_train)
print(f"Training cell count: {training_cell_count}")

## Define selected_cell_label

In [ ]:
# Get the cell-ID from cell_inventory
selected_cell_label = "2017-05-12_5_4C-70per_3C_CH17"

# Create a subfolder to store fig output
# corresponding to each cell-index
selected_cell_artifacts_dir = bconf.artifacts_output_dir(
    selected_cell_label)

# Load benchmarking dataset

In [ ]:
# Import the BenchDB class
# Load only the dataset based on the selected cell
benchdb = BenchDB(
    db_filepath,
    selected_cell_label)

# load the benchmarking dataset
df_selected_cell = benchdb.load_benchmark_dataset(
    dataset_type="train")

## Drop labels

In [ ]:
if df_selected_cell is not None:

    filter_col = [
        "cell_index",
        "cycle_index",
        "discharge_capacity",
        "voltage"]

    # Drop true labels from the benchmarking dataset
    # and filter for selected columns only
    df_selected_cell_without_labels = benchdb.drop_labels(
        df_selected_cell,
        filter_col)

    # print a subset of the dataframe
    # for diagnostics running in terminals
    print(df_selected_cell_without_labels.head(10).to_markdown())
    print("*"*100)

## Plot cycle data without labels

In [ ]:
# If the true outlier cycle index is not known,
# cycling data will be plotted without labels
benchdb.plot_cycle_data(
    df_selected_cell_without_labels)

output_fig_filename = (
    "cycle_data_without_labels_"
    + selected_cell_label
    + ".png")

fig_output_path = (
    selected_cell_artifacts_dir
    .joinpath(output_fig_filename))

plt.savefig(
    fig_output_path,
    dpi=600,
    bbox_inches="tight")

plt.show()

# Features transformation

## Load the training features dataset

In [ ]:
# Define the filepath to ``train_features_severson.db``
# osbad/database/train_features_severson.db
db_features_filepath = (
    DB_DIR.joinpath(
        "train_features_severson.db"))

# Load only the training features dataset
df_features_per_cell = benchdb.load_features_db(
    db_features_filepath,
    dataset_type="train")

unique_cycle_count = (
    df_features_per_cell["cycle_index"].unique())

In [ ]:
df_features_per_cell

# Hyperparameter tuning

## Create optuna study

In [ ]:
hp_config_lof = {
    "contamination": {"low": 0.0, "high": 0.5},
    "n_neighbors": {"low": 10, "high": 50},
    "leaf_size": {"low": 10, "high": 100},
    "metric": ["minkowski","euclidean","manhattan"],
    "threshold": {"low": 0.0, "high": 1.0}
}

lof_hp_config_filepath = (
    Path.cwd()
    .parents[3]
    .joinpath(
        "machine_learning",
        "hp_config_schema",
        "severson_hp_config",
        "lof_hp_config.json"))

bconf.create_json_hp_config(
    lof_hp_config_filepath,
    hp_dict=hp_config_lof)

bconf.load_json_hp_config(
    lof_hp_config_filepath)

In [ ]:
# Reload the hp module to refresh in-memory variables
# especially after updating parameters
from importlib import reload
reload(hp)

# Check if the schema in the script has been updated 
# based on the current constraints specified 
# from the notebook
print("Current hyperparameter config:")
print(hp._LOF_HP_CONFIG)
print("-"*70)

# Instantiate an optuna study for lof model
sampler = optuna.samplers.TPESampler(seed=42)

selected_feature_cols = (
    "cycle_index",
    "log_max_diff_dQ",
    "log_max_diff_dV")

lof_study = optuna.create_study(
    study_name="lof_hyperparam",
    sampler=sampler,
    directions=["minimize","maximize"])

lof_study.optimize(
    lambda trial: hp.objective(
        trial,
        model_id="lof",
        df_feature_dataset=df_features_per_cell,
        selected_feature_cols=selected_feature_cols,
        #df_benchmark_dataset=df_selected_cell,
        selected_cell_label=selected_cell_label),
    n_trials=100)

## Aggregate best trials

In [ ]:
schema_lof = {
    "contamination": "median",
    "n_neighbors": "median_int",
    "leaf_size": "median_int",
    "metric": "mode",
    "threshold": "median",
}

trade_off_trials_list = hp.trade_off_trials_detection_test(
    lof_study, 
    #window_size=7
    )

df_lof_hyperparam = hp.aggregate_best_trials(
    trade_off_trials_list,
    cell_label=selected_cell_label,
    model_id="lof",
    schema=schema_lof)

df_lof_hyperparam

## Plot pareto front

In [ ]:
hp.plot_proxy_pareto_front(
    lof_study,
    trade_off_trials_list,
    selected_cell_label,
    fig_title="Local Outlier Factor (LOF) Pareto Front")

plt.show()

## Export current hyperparameters to CSV 

In [ ]:
# Export current hyperparameters to CSV
hyperparam_filepath =  Path.cwd().joinpath(
    "ml_04_lof_hyperparam_proxy_severson.csv")

hp.export_current_hyperparam(
    df_lof_hyperparam,
    selected_cell_label,
    export_csv_filepath=hyperparam_filepath,
    if_exists="replace")

# Train model with best trial parameters

## Load best trial parameters from CSV output

In [ ]:
# Test reading from exported metrics
df_hyperparam_from_csv = pd.read_csv(hyperparam_filepath)

df_param_per_cell = df_hyperparam_from_csv[
    df_hyperparam_from_csv["cell_index"] == selected_cell_label]
df_param_per_cell

## Create a dict for best trial parameters

In [ ]:
param_dict = df_param_per_cell.iloc[0].to_dict()
pprint.pp(param_dict)

## Run the model with best trial parameters

In [ ]:
cfg = hp.MODEL_CONFIG["lof"]

selected_feature_cols = (
    "log_max_diff_dQ",
    "log_max_diff_dV")

runner = ModelRunner(
    cell_label=selected_cell_label,
    df_input_features=df_features_per_cell,
    selected_feature_cols=selected_feature_cols
)

Xdata = runner.create_model_x_input()

model = cfg.model_param(param_dict)
print(model)
model.fit(Xdata)
proba = model.predict_proba(Xdata)

pred_outlier_indices, pred_outlier_score = runner.pred_outlier_indices_from_proba(
    proba=proba,
    threshold=param_dict["threshold"],
    outlier_col=cfg.proba_col
)

pred_outlier_indices, pred_outlier_score

## Get df_outliers_pred

In [ ]:
df_outliers_pred = (df_features_per_cell[
    df_features_per_cell["cycle_index"]
    .isin(pred_outlier_indices)].copy())

df_outliers_pred["outlier_prob"] = pred_outlier_score
df_outliers_pred

## Predict anomaly score map

In [ ]:
axplot = runner.predict_anomaly_score_map(
    selected_model=model,
    model_name="Local Outlier Factor (LOF)",
    xoutliers=df_outliers_pred["log_max_diff_dQ"],
    youtliers=df_outliers_pred["log_max_diff_dV"],
    pred_outliers_index=pred_outlier_indices,
    threshold=param_dict["threshold"]
)

axplot.set_xlabel(
    r"$\log(\Delta Q_\textrm{scaled,max,cyc)}\;\textrm{[Ah]}$",
    fontsize=12)
axplot.set_ylabel(
    r"$\log(\Delta V_\textrm{scaled,max,cyc})\;\textrm{[V]}$",
    fontsize=12)

output_fig_filename = (
    "lof_"
    + selected_cell_label
    + ".png")

fig_output_path = (
    selected_cell_artifacts_dir
    .joinpath(output_fig_filename))

plt.savefig(
    fig_output_path,
    dpi=600,
    bbox_inches="tight")

plt.show()

# Model performance evaluation

In [ ]:
df_eval_outlier = modval.evaluate_pred_outliers(
    df_benchmark=df_selected_cell,
    outlier_cycle_index=pred_outlier_indices)

## Confusion matrix

In [ ]:
axplot = modval.generate_confusion_matrix(
    y_true=df_eval_outlier["true_outlier"],
    y_pred=df_eval_outlier["pred_outlier"])

axplot.set_title(
    "Local Outlier Factor (LOF)",
    fontsize=16)

output_fig_filename = (
    "conf_matrix_lof_"
    + selected_cell_label
    + ".png")

fig_output_path = (
    selected_cell_artifacts_dir
    .joinpath(output_fig_filename))

plt.savefig(
    fig_output_path,
    dpi=600,
    bbox_inches="tight")

plt.show()

## Evaluate model performance

In [ ]:
df_current_eval_metrics = modval.eval_model_performance(
    model_name="lof",
    selected_cell_label=selected_cell_label,
    df_eval_outliers=df_eval_outlier)

df_current_eval_metrics

## Export model performance metrics to CSV output

In [ ]:
# Export current metrics to CSV
hyperparam_eval_filepath =  Path.cwd().joinpath(
    "eval_metrics_hp_single_cell_severson.csv")

hp.export_current_model_metrics(
    model_name="lof",
    selected_cell_label=selected_cell_label,
    df_current_eval_metrics=df_current_eval_metrics,
    export_csv_filepath=hyperparam_eval_filepath,
    if_exists="replace")

# Finally: check with true labels

## Plot cycle data with labels

In [ ]:
# Extract true outliers cycle index from benchmarking dataset
true_outlier_cycle_index = benchdb.get_true_outlier_cycle_index(
    df_selected_cell)
print(f"True outlier cycle index:")
print(true_outlier_cycle_index)

# Plot cell data with true anomalies
# If the true outlier cycle index is not known,
# cycling data will be plotted without labels
benchdb.plot_cycle_data(
    df_selected_cell_without_labels,
    true_outlier_cycle_index)

output_fig_filename = (
    "cycle_data_with_labels_"
    + selected_cell_label
    + ".png")

fig_output_path = (
    selected_cell_artifacts_dir.joinpath(output_fig_filename))

plt.savefig(
    fig_output_path,
    dpi=600,
    bbox_inches="tight")

plt.show()

## Calculate bubble size ratio

In [ ]:
# Calculate the bubble size ratio for plotting
df_bubble_size_dQ = bviz.calculate_bubble_size_ratio(
    df_variable=df_features_per_cell["max_diff_dQ"])

df_bubble_size_dV = bviz.calculate_bubble_size_ratio(
    df_variable=df_features_per_cell["max_diff_dV"])

bubble_size = (
    np.abs(df_bubble_size_dV)
    * np.abs(df_bubble_size_dQ))

## Plot the bubble chart and label the true outliers

In [ ]:
# Plot the bubble chart and label the outliers
axplot = bviz.plot_bubble_chart(
    xseries=df_features_per_cell["log_max_diff_dQ"],
    yseries=df_features_per_cell["log_max_diff_dV"],
    bubble_size=bubble_size,
    unique_cycle_count=unique_cycle_count,
    cycle_outlier_idx_label=true_outlier_cycle_index)

axplot.set_title(
    f"Cell {selected_cell_label}", fontsize=13)

axplot.set_xlabel(
    r"$\log(\Delta Q_\textrm{scaled,max,cyc)}\;\textrm{[Ah]}$",
    fontsize=12)
axplot.set_ylabel(
    r"$\log(\Delta V_\textrm{scaled,max,cyc})\;\textrm{[V]}$",
    fontsize=12)

output_fig_filename = (
    "log_bubble_plot_"
    + selected_cell_label
    + ".png")

fig_output_path = (
    selected_cell_artifacts_dir.joinpath(output_fig_filename))

plt.savefig(
    fig_output_path,
    dpi=200,
    bbox_inches="tight")

plt.show()